In [1]:
# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display, HTML
# from LazyQML.lazyqml.lazyqml import QuantumClassifier
import pandas as pd
from sklearn.model_selection import train_test_split

import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../lazyqml/')

# Add custom CSS for better aesthetics
display(HTML("""
<style>
    .widget-label {
        font-weight: bold;
        font-size: 14px;
    }
             
    .widget-checkbox {
        width: auto
    }

    .widget-text {
        width: auto
    }
    
    .widget-dropdown {
        width: auto
    }
    
    .widget-button {
        width: auto
    }
             
    .widget-vbox {
        background-color: #f9f9f9;
        padding: 15px;
        border-radius: 10px;
        box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);
        margin-bottom: 15px;
        width: auto
    }

</style>
"""))

In [2]:
output_display = widgets.Output()

# Widgets for user input with enhanced styles and multi-selection
epochs_widget = widgets.IntText(
    min=1, 
    max=1000, 
    step=1, 
    value=10, 
    description="Epochs:",
    indent=False
)

layers_widget = widgets.IntText(
    value=7.5,
    min=0,
    max=100,
    step=1,
    description='Layers:',
    disabled=False,
    indent=False
)

nqubits_widget = widgets.IntText(
    min=1,
    max=30, 
    step=1, 
    value=4, 
    description="Qubits:",
    indent=False
)

randomstate_widget = widgets.IntText(
    value=1234, 
    description="Seed:",
    indent=False
)

runs_widget = widgets.IntText(
    min=1,
    max=10, 
    step=1, 
    value=1, 
    description="Runs:",
    indent=False
)

dataset_widget = widgets.Dropdown(
    options=['Iris', 'Breast Cancer', 'Wine'],
    value='Iris',
    description='Dataset:',
    disabled=False,
)

# Cross validation options
cv_checkbox = widgets.Checkbox(
    value=False,
    description='Cross-Validation',
    disabled=False,
    indent=False
)

splits_widget = widgets.IntText(
    min=1,
    max=50, 
    step=1, 
    value=4, 
    description="Splits:",
    indent=False
)

repeats_widget = widgets.IntText(
    min=1,
    max=50, 
    step=1, 
    value=4, 
    description="Repeats:",
    indent=False
)

# # Classifier multi-selection using a Dropdown with multiple selection enabled
# classifiers_options = ['qsvm', 'qnn', 'qnnbag']
# classifiers_widget = widgets.SelectMultiple(
#     options=classifiers_options, value=['qsvm'], description='Classifiers:',
#     style={'description_width': '150px'}, layout=widgets.Layout(width='500px')
# )

# Models
qsvm_checkbox = widgets.Checkbox(
    value=False,
    description='QSVM',
    disabled=False,
    indent=False
)

qnn_checkbox = widgets.Checkbox(
    value=False,
    description='QNN',
    disabled=False,
    indent=False
)

qnn_bag_checkbox = widgets.Checkbox(
    value=False,
    description='QNN Bagging',
    disabled=False,
    indent=False
)

# Ansatzs
hp_checkbox = widgets.Checkbox(
    value=False,
    description='HCzRx',
    disabled=False,
    indent=False
)

tt_checkbox = widgets.Checkbox(
    value=False,
    description='Tree Tensor',
    disabled=False,
    indent=False
)

two_checkbox = widgets.Checkbox(
    value=False,
    description='Two Local',
    disabled=False,
    indent=False
)

hwe_checkbox = widgets.Checkbox(
    value=False,
    description='Hardware Efficient',
    disabled=False,
    indent=False
)

annular_checkbox = widgets.Checkbox(
    value=False,
    description='Annular',
    disabled=False,
    indent=False
)

# Embeddings
rx_checkbox = widgets.Checkbox(
    value=False,
    description='Rotation X',
    disabled=False,
    indent=False
)

ry_checkbox = widgets.Checkbox(
    value=False,
    description='Rotation Y',
    disabled=False,
    indent=False
)

rz_checkbox = widgets.Checkbox(
    value=False,
    description='Rotation Z',
    disabled=False,
    indent=False
)

zz_checkbox = widgets.Checkbox(
    value=False,
    description='ZZ',
    disabled=False,
    indent=False
)

amp_checkbox = widgets.Checkbox(
    value=False,
    description='Amplitude',
    disabled=False,
    indent=False
)

dense_checkbox = widgets.Checkbox(
    value=False,
    description='Dense Angle',
    disabled=False,
    indent=False
)

ho_checkbox = widgets.Checkbox(
    value=False,
    description='Higher Order',
    disabled=False,
    indent=False
)



# Ansatz multi-selection with Dropdown widget for intuitive multiple selection
# ansatzs_options = ['HPzRx', 'tree_tensor', 'two_local', 'hardware_efficient']
# ansatzs_widget = widgets.SelectMultiple(
#     options=ansatzs_options, value=['HPzRx'], description='Ansatzs:',
#     style={'description_width': '150px'}, layout=widgets.Layout(width='500px'))

# # Embeddings multi-selection using Dropdown
# embeddings_options = ['amplitude_embedding', 'ZZ_embedding', 'rx_embedding', 'rz_embedding', 'ry_embedding']
# embeddings_widget = widgets.SelectMultiple(
#     options=embeddings_options, value=['amplitude_embedding'], description='Embeddings:',
#     style={'description_width': '150px'}, layout=widgets.Layout(width='500px'))

# Verbose switch
verbose_widget = widgets.Checkbox(
    value=False, description="Verbose", indent=False)

# Number of samples as a percentage (from 0.0 to 1.0)
nsamples_widget = widgets.FloatSlider(
    value=1.0,
    min=0.0, 
    max=1.0, 
    step=0.01, 
    description="Samples (0, 1):",
    layout=widgets.Layout(width='90%'))

nfeatures_widget = widgets.FloatSlider(
    value=1.0,
    min=0.0, 
    max=1.0, 
    step=0.01, 
    description="Features (0, 1):",
    layout=widgets.Layout(width='90%'))


# Number of estimators (for QNN Bagging)
nestimators_widget = widgets.IntText(
    min=1, max=50, step=1, value=10, description="Estimators:")

# Groups of options
general_model_options_box = widgets.VBox([widgets.Label("General model options:"), epochs_widget, layers_widget, nqubits_widget, dataset_widget], layout=widgets.Layout(width='50%', margin='10pt'))
cv_options_box = widgets.VBox([widgets.Label("Cross-Validation options:"), cv_checkbox, splits_widget, repeats_widget], layout=widgets.Layout(width='50%', margin='10pt'))
classifiers_box = widgets.VBox([widgets.Label("Classifiers:"), qsvm_checkbox, qnn_checkbox, qnn_bag_checkbox], layout=widgets.Layout(width='50%', margin='10pt'))
ansatzs_box = widgets.VBox([widgets.Label("Ansatzs:"), hp_checkbox, tt_checkbox, two_checkbox, hwe_checkbox, annular_checkbox], layout=widgets.Layout(width='50%', margin='10pt'))
embeddings_box = widgets.VBox([widgets.Label("Embeddings:"), rx_checkbox, ry_checkbox, rz_checkbox, zz_checkbox, amp_checkbox, dense_checkbox, ho_checkbox], layout=widgets.Layout(width='50%', margin='10pt'))

bagging_box = widgets.VBox([widgets.Label("Bagging options:"), nestimators_widget, nsamples_widget, nfeatures_widget], layout=widgets.Layout(width='100%', margin='10pt'))
other_options_box = widgets.VBox([widgets.Label("Other options:"), runs_widget, randomstate_widget, verbose_widget], layout=widgets.Layout(width='100%', margin='10pt'))

#############

out_code = widgets.Textarea(disabled=False, layout=widgets.Layout(width='100%', height='250px'))

output_code_display = widgets.Output(style={'word-wrap': 'break-word'})
output_code_display.layout.display = 'flex'

# Generate code function
@output_code_display.capture(clear_output=True)
def generate_code(button):

    # General options
    epochs = epochs_widget.value
    layers = layers_widget.value
    qubits = nqubits_widget.value
    dataset_ix = dataset_widget.value
    test_size = 0.7

    # Cross validation
    cv = cv_checkbox.value
    splits = splits_widget.value
    repeats = repeats_widget.value

    # Classifiers
    qsvm = qsvm_checkbox.value
    qnn = qnn_checkbox.value
    qnn_bag = qnn_bag_checkbox.value

    # Ansatzs
    hczrx = hp_checkbox.value
    tree_tensor = tt_checkbox.value
    two_local = two_checkbox.value
    hardware_efficient = hwe_checkbox.value
    annular = annular_checkbox.value

    # Embeddings
    rx = rx_checkbox.value
    ry = ry_checkbox.value
    rz = rz_checkbox.value
    zz = zz_checkbox.value
    amplitude = amp_checkbox.value
    higher_order = ho_checkbox.value
    dense_angle = dense_checkbox.value


    # Bagging options
    estimators = nestimators_widget.value
    samples = nsamples_widget.value
    features = nfeatures_widget.value

    # Other options
    runs = runs_widget.value
    seed = randomstate_widget.value
    verbose = verbose_widget.value

    selected_embeddings = []
    selected_ansatzs = []
    selected_classifiers = []

    if qsvm: selected_classifiers.append("Model.QSVM")
    if qnn: selected_classifiers.append("Model.QNN")
    if qnn_bag: selected_classifiers.append("Model.QNN_BAG")

    if hczrx: selected_ansatzs.append("Ansatzs.HCZRX")
    if tree_tensor: selected_ansatzs.append("Ansatzs.TREE_TENSOR")
    if two_local: selected_ansatzs.append("Ansatzs.TWO_LOCAL")
    if hardware_efficient: selected_ansatzs.append("Ansatzs.HARDWARE_EFFICIENT")
    if annular: selected_ansatzs.append("Ansatzs.ANNULAR")

    if rx: selected_embeddings.append("Embedding.RX")
    if ry: selected_embeddings.append("Embedding.RY")
    if rz: selected_embeddings.append("Embedding.RZ")
    if zz: selected_embeddings.append("Embedding.ZZ")
    if amplitude: selected_embeddings.append("Embedding.AMP")
    if higher_order: selected_embeddings.append("Embedding.HIGHER_ORDER")
    if dense_angle: selected_embeddings.append("Embedding.DENSE_ANGLE")

    selected_classifiers = "{" + ", ".join(selected_classifiers) + "}"
    selected_ansatzs = "{" + ", ".join(selected_ansatzs) + "}"
    selected_embeddings = "{" + ", ".join(selected_embeddings) + "}"

    datasets = {
        "Iris": "iris",
        "Breast Cancer": "breast_cancer",
        "Wine": "wine"
    }

    selected_dataset = datasets[dataset_ix]

    classifier_params = {
        "randomstate": seed,
        "epochs": epochs,
        "numLayers": layers,
        "runs": runs,
        "verbose": verbose,
        "maxSamples": samples,
        "numPredictors": estimators,
        "features": features,
        "embeddings": selected_embeddings,
        "ansatzs": selected_ansatzs,
        "classifiers": selected_classifiers,
        "nqubits": {qubits}
    }

    fit_params = {
        "X": "X",
        "y": "y"
    }

    # Code snippet preparation
    fit = ""
    if cv:
        fit_params["n_splits"] = splits
        fit_params["n_repeats"] = repeats

        fit += "classifier.repeated_cross_validation("
    else:
        fit_params["test_size"] = test_size
        fit += "classifier.fit("

    for k, v in fit_params.items():
        fit += f"{k}={v},"

    fit = fit[:-1] + ")"

    imports = \
    f"from sklearn.datasets import load_{selected_dataset}\nfrom lazyqml.lazyqml import *"

    data_loading = \
    f"data = load_{selected_dataset}()\nX = data.data\ny = data.target"

    classifier_params_code = "classifier = QuantumClassifier("
    for k, v in classifier_params.items():
        classifier_params_code += f"{k}={v}, "
    classifier_params_code = classifier_params_code[:-2] + ")"

    # Code snippet
    code_snippet = "\n\n".join([imports, data_loading, classifier_params_code, fit])

    print(code_snippet)

    out_code.value = code_snippet

    return code_snippet

#############

# Generate code button
generate_code_button = widgets.Button(
    description='Generate code',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

generate_code_button.on_click(generate_code)

# Function to display widgets in a nice box layout (stack widgets vertically)
# @output_display.capture()
def display_widgets():
    row_0 = widgets.HBox([general_model_options_box, cv_options_box])
    row_1 = widgets.HBox([classifiers_box, ansatzs_box, embeddings_box])
    row_2 = widgets.HBox([bagging_box, other_options_box])
    
    widget_box = widgets.VBox([
        widgets.HTML("<h3>Quantum Classifier Configuration</h3>"),
        row_0, 
        row_1,
        row_2,
        generate_code_button,
        out_code
        # output_code_display
    ], layout=widgets.Layout(border='solid 1px #cccccc', border_radius='10px'))
    
    display(widget_box)


In [3]:
# Display
display_widgets()

